# ASKLB Jupyter Notebook 
### Note: If running on new Google Colab instance, run the three lines below to download dependencies:
!apt-get install build-essential swig <br>
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install <br>
!pip install auto-sklearn

### Colab Link
https://colab.research.google.com/github/KordingLab/ASKLB/blob/master/notebooks/ASKLB_Notebook.ipynb

Uncomment and run this code if on Colab and need to install dependencies for Auto-Sklearn

In [6]:
# !apt-get install build-essential swig 
# !curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install 
# !pip install auto-sklearn

In [7]:
import autosklearn.classification
import autosklearn.regression
import sklearn.model_selection
import numpy
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label
import warnings
import threading
import time
import os, sys
import sklearn.metrics as metrics

### Load X and y datasets
Should have names automl_X.csv and automl_Y.csv respectively (will see if I can use a file upload widget)

In [8]:
X = numpy.loadtxt('automl_X.csv', delimiter=',')
y = numpy.loadtxt('automl_y.csv', delimiter=',')
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

In [9]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def work(progress):
        for i in range(int(progress.max/5)):
            time.sleep(5)
            progress.value = progress.value+5

def on_button_clicked(b):        
    models_output.clear_output()
    metrics_output.clear_output()
    b.disabled=True
    final_runtime_value_seconds = runtime_widget.value * 60
    progress.value = 0    
    progress.max=final_runtime_value_seconds
    with output:
        print("AUTOML FITTING STARTED, FITTING TIME IS ", int(final_runtime_value_seconds/60), " MINUTES")
        
    automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task = final_runtime_value_seconds)
    thread = threading.Thread(target=work, args=(progress,))    
    thread.start()
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with HiddenPrints():
            automl.fit(X_train, y_train)
    
    with output:
        print("FITTING COMPLETED WITH FITTING TIME PARAMETER AS ", int(final_runtime_value_seconds/60), " MINUTES")
        
    with metrics_output:            
        y_train_hat = automl.predict(X_train)
        train_accuracy_score = sklearn.metrics.accuracy_score(y_train, y_train_hat)
        print("Train Accuracy Score: ", train_accuracy_score)
        
        
        y_test_hat = automl.predict(X_test)
        test_accuracy_score = sklearn.metrics.accuracy_score(y_test, y_test_hat)
        print("Test Accuracy Score: ", test_accuracy_score)
        
        fpr_train, tpr_train, threshold_train = metrics.roc_curve(y_train, y_train_hat)
        roc_auc_train = metrics.auc(fpr_train, tpr_train)
        
        print("Train ROC AUC: ", roc_auc_train)      
        
        fpr_test, tpr_test, threshold_test = metrics.roc_curve(y_test, y_test_hat)
        roc_auc_test = metrics.auc(fpr_test, tpr_test)
        
        print("Test ROC AUC: ", roc_auc_test)
        
    with models_output:
        print("MODELS:")
        print(automl.get_models_with_weights())
        
    b.disabled=False

In [10]:
items_layout = widgets.Layout( width='auto')

runtime_widget = widgets.IntSlider(
    value=15,
    min=1,
    max=60)

progress = widgets.IntProgress(value=0, min=0, description="Progress")

button = widgets.Button(description="Fit Data to AutoML Model", layout = items_layout, button_style='primary')
button.on_click(on_button_clicked)

runtime_slider = HBox([Label('AutoML Runtime (minutes)'), runtime_widget])

output = widgets.Output(layout={'border': '1px solid black'})
metrics_output = widgets.Output()
models_output = widgets.Output()
models_accordian = widgets.Accordion(children=[metrics_output, models_output])
models_accordian.set_title(0, 'Performance Metrics')
models_accordian.set_title(1, 'Models and Weights Data')

### AutoML WIDGET

In [11]:
widget_items = [runtime_slider, button, progress, output, models_accordian]

box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='65%')
box = widgets.Box(children=widget_items, layout=box_layout)
box

Box(children=(HBox(children=(Label(value='AutoML Runtime (minutes)'), IntSlider(value=15, max=60, min=1))), Bu…